In [98]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk import ngrams
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import numpy as np
import torch
import torch.nn as nn


Imagine you have to get a sentiment score from a movie review.

Simple way: two lists of words, one positive, one negative. Don't even need a model, if the number of positive words is higher, it is a positive sentiment for the movie review. 

In [99]:
def find_sentiment(sentence, pos, neg):
    sentence = sentence.split()
    
    # make a set of the words in the sentence
    sentence = set(sentence)
    
    num_common_pos = len(sentence.intersection(pos))
    
    num_common_neg = len(sentence.intersection(neg))
    
    if num_common_pos > num_common_neg:
        return "positive"
    if num_common_pos < num_common_neg:
        return "negative"
    return "neutral"
    

In [100]:
sentence = "hi, how are you?"

In [101]:
sentence.split()

['hi,', 'how', 'are', 'you?']

#### Bag of words
We create a huge sparse matrix that stores counts of all the words in our corpus.

In [102]:
# create a corpus of sentences
corpus = [
"hello, how are you?",
"im getting bored at home. And you? What do you think?",
"did you know about counts",
"let's see if this works!",
"YES!!!!"
]

# initialize CountVectorizer
ctv = CountVectorizer()

# fit the vectorizer on corpus
ctv.fit(corpus)
corpus_transformed = ctv.transform(corpus)

print(corpus_transformed)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 25 stored elements and shape (5, 23)>
  Coords	Values
  (0, 2)	1
  (0, 9)	1
  (0, 11)	1
  (0, 22)	1
  (1, 1)	1
  (1, 3)	1
  (1, 4)	1
  (1, 7)	1
  (1, 8)	1
  (1, 10)	1
  (1, 13)	1
  (1, 17)	1
  (1, 19)	1
  (1, 22)	2
  (2, 0)	1
  (2, 5)	1
  (2, 6)	1
  (2, 14)	1
  (2, 22)	1
  (3, 12)	1
  (3, 15)	1
  (3, 16)	1
  (3, 18)	1
  (3, 20)	1
  (4, 21)	1


In [103]:
# create a corpus of sentences
corpus = [
"hello, how are you?",
"im getting bored at home. And you? What do you think?",
"did you know about counts",
"let's see if this works!",
"YES!!!!"
]
# initialize CountVectorizer with word_tokenize from nltk
# as the tokenizer
ctv = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)
# fit the vectorizer on corpus
ctv.fit(corpus)
corpus_transformed = ctv.transform(corpus)
print(ctv.vocabulary_)

{'hello': 14, ',': 2, 'how': 16, 'are': 7, 'you': 27, '?': 4, 'im': 18, 'getting': 13, 'bored': 9, 'at': 8, 'home': 15, '.': 3, 'and': 6, 'what': 24, 'do': 12, 'think': 22, 'did': 11, 'know': 19, 'about': 5, 'counts': 10, 'let': 20, "'s": 1, 'see': 21, 'if': 17, 'this': 23, 'works': 25, '!': 0, 'yes': 26}


Create a sparse matrix by using all the sentences in IMDB sataset and build a model. Use accuracy as the metric.

In [104]:
if __name__ == "__main__":
    # read the training data
    df = pd.read_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb.csv")

    # map positive to 1 and negative to 0
    df.sentiment = df.sentiment.apply(
    lambda x: 1 if x == "positive" else 0)

    df["kfold"] = -1

    df = df.sample(frac=1).reset_index(drop=True)

    # fetch labels
    y = df.sentiment.values

    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=5)

    # fill the new kfold column
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f

    for fold_ in range(5):
        train_df = df[df.kfold != fold_].reset_index(drop=True)
        test_df = df[df.kfold == fold_].reset_index(drop=True)

        # initialize CountVectorizer with NLTK's word_tokenize
        # function as the tokenizer
        ctv = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)

        # fit count vectorizer on training data reviews
        ctv.fit(train_df.review)

        # transform training and validation data reviews
        xtrain = ctv.transform(train_df.review)
        xtest = ctv.transform(test_df.review)

        # initialize logistic regression model
        model = linear_model.LogisticRegression()

        # fit the model on training data reviews and sentiment
        model.fit(xtrain, train_df.sentiment)

        # make predictions on test data
        # threshold for predictions is 0.5
        preds = model.predict(xtest)

        # calculate accuracy
        accuracy = metrics.accuracy_score(test_df.sentiment, preds)

        print(f"Fold: {fold_}")
        print(f"Accuracy = {accuracy}")
        print("")

c:\Users\Chand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold: 0
Accuracy = 0.8943



c:\Users\Chand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold: 1
Accuracy = 0.8843



c:\Users\Chand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold: 2
Accuracy = 0.8895



c:\Users\Chand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold: 3
Accuracy = 0.8875

Fold: 4
Accuracy = 0.9005



c:\Users\Chand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


We are at the 89 percent accuracy but the model took a long time to train. Can improve time by using naive bayes classifier.

In [105]:
if __name__ == "__main__":
    # Read the training data
    df = pd.read_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb.csv")

    # Map positive to 1 and negative to 0
    df.sentiment = df.sentiment.apply(lambda x: 1 if x == "positive" else 0)

    # Add and shuffle kfold column
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)

    # Fetch labels
    y = df.sentiment.values

    # Initiate StratifiedKFold
    kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Assign folds
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f

    # 5-fold cross-validation
    for fold_ in range(5):
        train_df = df[df.kfold != fold_].reset_index(drop=True)
        test_df = df[df.kfold == fold_].reset_index(drop=True)

        # Initialize CountVectorizer with NLTK's word_tokenize function
        ctv = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)

        # Fit CountVectorizer on training data reviews
        ctv.fit(train_df.review)

        # Transform training and validation data reviews
        xtrain = ctv.transform(train_df.review)
        xtest = ctv.transform(test_df.review)

        # Initialize Multinomial Naive Bayes model
        model = naive_bayes.MultinomialNB()

        # Fit the model on training data reviews and sentiment
        model.fit(xtrain, train_df.sentiment)

        # Make predictions on test data
        preds = model.predict(xtest)

        # Calculate accuracy
        accuracy = metrics.accuracy_score(test_df.sentiment, preds)

        print(f"Fold: {fold_}")
        print(f"Accuracy = {accuracy}")
        print("")

Fold: 0
Accuracy = 0.8494

Fold: 1
Accuracy = 0.8412

Fold: 2
Accuracy = 0.843

Fold: 3
Accuracy = 0.8442

Fold: 4
Accuracy = 0.8423



### N-grams: combinations of words in order. And may increase model accuracy.

In [106]:
# let's see 3 grams
N = 3
# input sentence
sentence = "hi, how are you?"
# tokenized sentence
tokenized_sentence = word_tokenize(sentence)
# generate n_grams
n_grams = list(ngrams(tokenized_sentence, N))
print(n_grams)

[('hi', ',', 'how'), (',', 'how', 'are'), ('how', 'are', 'you'), ('are', 'you', '?')]


In [107]:
tfidf_vec = TfidfVectorizer(
tokenizer=word_tokenize,
token_pattern=None,
ngram_range=(1, 3)
)

In [108]:
if __name__ == "__main__":
    # Read the training data
    df = pd.read_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb.csv")

    # Map positive to 1 and negative to 0
    df.sentiment = df.sentiment.apply(lambda x: 1 if x == "positive" else 0)

    # Add and shuffle kfold column
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)

    # Fetch labels
    y = df.sentiment.values

    # Initiate StratifiedKFold
    kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Assign folds
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f

    # 5-fold cross-validation
    for fold_ in range(5):
        train_df = df[df.kfold != fold_].reset_index(drop=True)
        test_df = df[df.kfold == fold_].reset_index(drop=True)

        # Initialize TfidfVectorizer with NLTK's word_tokenize function
        tfidf_vec = TfidfVectorizer(
            tokenizer=word_tokenize,
            token_pattern=None,
            ngram_range=(1, 3)  # Includes unigrams, bigrams, and trigrams
        )

        # Fit TfidfVectorizer on training data reviews
        tfidf_vec.fit(train_df.review)

        # Transform training and validation data reviews
        xtrain = tfidf_vec.transform(train_df.review)
        xtest = tfidf_vec.transform(test_df.review)

        # Initialize Multinomial Naive Bayes model
        model = naive_bayes.MultinomialNB()

        # Fit the model on training data reviews and sentiment
        model.fit(xtrain, train_df.sentiment)

        # Make predictions on test data
        preds = model.predict(xtest)

        # Calculate accuracy
        accuracy = metrics.accuracy_score(test_df.sentiment, preds)

        print(f"Fold: {fold_}")
        print(f"Accuracy = {accuracy}")
        print("")


Fold: 0
Accuracy = 0.8878

Fold: 1
Accuracy = 0.8904

Fold: 2
Accuracy = 0.8895

Fold: 3
Accuracy = 0.895

Fold: 4
Accuracy = 0.8864



#### Stemming and lemmatization: reduce a word to its smallest form

In [109]:
lemmatizer = WordNetLemmatizer()

# initialize Snowball Stemmer
stemmer = SnowballStemmer("english")

words = ["worders", "drove", "cats", "running"]

for word in words:
    print(f"word={word}")
    print(f"stemmed_word={stemmer.stem(word)}")
    print(f"lemma={lemmatizer.lemmatize(word)}")
    print("")

word=worders
stemmed_word=worder
lemma=worders

word=drove
stemmed_word=drove
lemma=drove

word=cats
stemmed_word=cat
lemma=cat

word=running
stemmed_word=run
lemma=running



See if adding stemming can increase accuracy

In [110]:
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

# Function to preprocess text using lemmatization and stemming
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Lemmatize and stem each token
    processed_tokens = [stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens]
    # Join the processed tokens back into a single string
    return ' '.join(processed_tokens)

if __name__ == "__main__":
    # Read the training data
    df = pd.read_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb.csv")

    # Map positive to 1 and negative to 0
    df.sentiment = df.sentiment.apply(lambda x: 1 if x == "positive" else 0)

    # Add and shuffle kfold column
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)

    # Fetch labels
    y = df.sentiment.values

    # Initiate StratifiedKFold
    kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Assign folds
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f

    # 5-fold cross-validation
    for fold_ in range(5):
        train_df = df[df.kfold != fold_].reset_index(drop=True)
        test_df = df[df.kfold == fold_].reset_index(drop=True)

        # Preprocess the review text using lemmatization and stemming
        train_df["review"] = train_df["review"].apply(preprocess_text)
        test_df["review"] = test_df["review"].apply(preprocess_text)

        # Initialize TfidfVectorizer with NLTK's word_tokenize function
        tfidf_vec = TfidfVectorizer(
            tokenizer=word_tokenize,
            token_pattern=None,
            ngram_range=(1, 3)  # Includes unigrams, bigrams, and trigrams
        )

        # Fit TfidfVectorizer on training data reviews
        tfidf_vec.fit(train_df.review)

        # Transform training and validation data reviews
        xtrain = tfidf_vec.transform(train_df.review)
        xtest = tfidf_vec.transform(test_df.review)

        # Initialize Multinomial Naive Bayes model
        model = naive_bayes.MultinomialNB()

        # Fit the model on training data reviews and sentiment
        model.fit(xtrain, train_df.sentiment)

        # Make predictions on test data
        preds = model.predict(xtest)

        # Calculate accuracy
        accuracy = metrics.accuracy_score(test_df.sentiment, preds)

        print(f"Fold: {fold_}")
        print(f"Accuracy = {accuracy}")
        print("")


Fold: 0
Accuracy = 0.8922

Fold: 1
Accuracy = 0.888

Fold: 2
Accuracy = 0.8964

Fold: 3
Accuracy = 0.8886

Fold: 4
Accuracy = 0.8907



#### Topic extraction
- Can be done using non-negaitve matric factorization (NMF) or latent semantic analysis(LSA).
This can reduce the data to a given number of components.

- TF-IDF (Term Frequency-Inverse Document Frequency) features are numerical representations of words or phrases in a text corpus that reflect their importance in a document relative to the entire corpus. These features are commonly used in natural language processing (NLP) and machine learning tasks involving text data.

In [111]:
# let's read only 10k samples from the data.
corpus = pd.read_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb.csv", nrows=10000)
corpus = corpus.review.values

# initialize TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)

# fit the vectorizer on corpus
tfv.fit(corpus)

# transform the corpus
corpus_transformed = tfv.transform(corpus)

# initialize SVD with 10 components
svd = decomposition.TruncatedSVD(n_components=10)

# fit SVD
corpus_svd = svd.fit(corpus_transformed)

sample_index = 0
feature_scores = dict(
    zip(
        tfv.get_feature_names_out(),
        corpus_svd.components_[sample_index]
    )
)

# once we have the feature scores, we can sort them in descending order
N = 5
print(sorted(feature_scores, key=feature_scores.get, reverse=True)[:N])

['the', ',', '.', 'a', 'and']


#### clean text data

In [112]:
def clean_text(s):
    # remove punctuation
    s = s.split()
    
    # join tokens by single space. this will remove all kinds of weird space characters
    s = " ".join(s)
    
    # remove all punctuation
    s = re.sub(f"[{re.escape(string.punctuation)}]", "", s)
    
    return s

# This function will convert a string like “hi, how are you????” to “hi how are you”.

In [113]:
corpus = pd.read_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb.csv", nrows=10000)
corpus.loc[:, "review"] = corpus.review.apply(clean_text)
corpus = corpus.review.values


# then rerun the code for training the model
# initialize TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)

# fit the vectorizer on corpus
tfv.fit(corpus)

# transform the corpus
corpus_transformed = tfv.transform(corpus)

# initialize SVD with 10 components
svd = decomposition.TruncatedSVD(n_components=10)

# fit SVD
corpus_svd = svd.fit(corpus_transformed)

sample_index = 0
feature_scores = dict(
    zip(
        tfv.get_feature_names_out(),
        corpus_svd.components_[sample_index]
    )
)

# once we have the feature scores, we can sort them in descending order
N = 5
print(sorted(feature_scores, key=feature_scores.get, reverse=True)[:N])

['the', 'a', 'and', 'of', 'to']


#### Word Embeddings
- Meaning: representation of words into vectors.
- Word2Vec is the oldest approach to convert words into vectors.
- Continuous Bag of Words: train a network to predict a missing word by using all the words around and during this process, the network will learn and update embeddings for all the words involved.
- Skip-gram model: take one word and predict the context words instead.

In [114]:
## Creat a function that take all the individual word vectors in a give sentences and create a normalized vector from all the word vectors of the tokens.

def sentence_to_vec(s, embedding_dict, stop_words, tokenizer):
    words = str(s).lower()
    words = tokenizer(words)
    
    # remove stop words
    words = [w for w in words if not w in stop_words]
    
    # keep only alpha numeric tokens
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        if w in embedding_dict:
            M.append(embedding_dict[w])
    if len(M) == 0:
        return np.zeros(300)
    M = np.array(M)
    
    # convert the list of embeddings to array
    v = M.sum(axis=0)
    
    # return normalized vector
    return v / np.sqrt((v ** 2).sum())

In [115]:
## application of the function
# Sample embedding dictionary
embedding_dict = {
    "the": np.random.rand(300),
    "cat": np.random.rand(300),
    "sat": np.random.rand(300),
    "on": np.random.rand(300),
    "mat": np.random.rand(300)
}

# Stop words
stop_words = {"the", "on"}

# Sample sentence
sentence = "The cat sat on the mat."

# Convert sentence to vector
vector = sentence_to_vec(sentence, embedding_dict, stop_words, word_tokenize)
print("Embedding vector for the sentence:")
print(vector)


Embedding vector for the sentence:
[0.04546079 0.06171939 0.05681235 0.09107364 0.04482545 0.07707392
 0.05248489 0.02372561 0.07575552 0.04421887 0.04683005 0.09040362
 0.06337044 0.07052759 0.03960564 0.06728026 0.04897506 0.06118425
 0.07011926 0.07556736 0.05487185 0.06563027 0.02030972 0.07034759
 0.07748291 0.03972214 0.08640887 0.01519241 0.04074792 0.04309249
 0.04309843 0.08706945 0.04783806 0.03630182 0.05519518 0.07638176
 0.06668437 0.06510008 0.06726166 0.04505232 0.02989269 0.04951909
 0.04510504 0.0422993  0.06033995 0.02451907 0.0543995  0.0161094
 0.02797804 0.0334433  0.07792725 0.07308441 0.0216232  0.05297646
 0.0503801  0.017344   0.05875385 0.02622825 0.04870248 0.0581891
 0.05735836 0.04680813 0.06153573 0.05533239 0.03790741 0.05766763
 0.04834462 0.06681266 0.0397839  0.05850705 0.07629182 0.04673489
 0.03562533 0.08619944 0.06604801 0.06143598 0.05828666 0.07390043
 0.04049738 0.05028619 0.07969187 0.05435471 0.04871565 0.03611747
 0.093696   0.04226273 0.0951

In [116]:
sentence1 = "I love programming."
sentence2 = "Coding is my passion."

# Generate vectors
vec1 = sentence_to_vec(sentence1, embedding_dict, stop_words, word_tokenize)
vec2 = sentence_to_vec(sentence2, embedding_dict, stop_words, word_tokenize)

# Compute cosine similarity
cosine_similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
print("Cosine Similarity:", cosine_similarity)


Cosine Similarity: nan


C:\Users\Chand\AppData\Local\Temp\ipykernel_23036\888311115.py:9: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


#### Text data is just like time series data. 
We can use Long short term memory (LSTM) or Gated Recurrent Units (GRU) or CNN.

In [117]:
if __name__ == "__main__":
    # read training data
    df = pd.read_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb.csv")
    
    df.sentiment = df.sentiment.apply(lambda x: 1 if x == "positive" else 0)
    
    df["kfold"] = -1
    
    df = df.sample(frac=1).reset_index(drop=True)   
    
    y = df.sentiment.values
    
    kf = model_selection.StratifiedKFold(n_splits=5)
    
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f
        
    df.to_csv(r"C:\Users\Chand\OneDrive\桌面\ML Project\NLP\imdb_folds.csv", index=False)

In [118]:
class IMDBDataset:
    def __init__(self, reviews, targets):
        """
        :param reviews: list of strings
        :param targets: list of integers
        """
        self.reviews = reviews
        self.targets = targets
        
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = self.reviews[item]
        target = self.targets[item]
        
        return {
            "review": torch.tensor(review, dtype=torch.long),
            "target": torch.tensor(target, dtype=torch.float)
        }

In [119]:
# Example reviews and targets
example_reviews = [
    "The movie was fantastic and I loved it.",
    "It was a terrible movie, not worth watching.",
    "An average film with decent acting and plot.",
    "Absolutely loved the direction and storyline.",
    "The movie was too slow and boring."
]
example_targets = [1, 0, 1, 1, 0]

# Simple word-to-index tokenizer
word_to_index = {
    word: i + 1 for i, word in enumerate(set(" ".join(example_reviews).split()))
}

def tokenize(review):
    return [word_to_index[word] for word in review.split()]

# Updated IMDBDataset class
class IMDBDataset:
    def __init__(self, reviews, targets):
        self.reviews = [tokenize(review) for review in reviews]
        self.targets = targets
        
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = self.reviews[item]
        target = self.targets[item]
        
        return {
            "review": torch.tensor(review, dtype=torch.long),
            "target": torch.tensor(target, dtype=torch.float)
        }

# Create dataset instance
dataset = IMDBDataset(example_reviews, example_targets)

# Access a sample
sample = dataset[0]
print(sample)


{'review': tensor([13,  4,  8, 24,  6, 27, 22, 21]), 'target': tensor(1.)}


In [120]:
class LSTM(nn.Module):
    def __init__(self, embedding_matrix):
        super(LSTM, self).__init__()
        
        # number of words = number of rows in the embedding matrix
        num_words = embedding_matrix.shape[0]
        
        #dimension of the embedding matrix = number of columns in the embedding matrix
        embed_dim = embedding_matrix.shape[1]
        
        # define an input embedding layer
        self.embedding = nn.Embedding(
            num_embeddings=num_words,
            embedding_dim=embed_dim
        )
        
        # set the weights of the embedding layer
        self.embedding.weight = nn.Parameter(
            torch.tensor(embedding_matrix, dtype=torch.float32)
        )
        
        # don't update the weights of the embedding layer
        self.embedding.weight.requires_grad = False
        
        # LSTM layer
        self.lstm = nn.LSTM(
            embed_dim,
            128,
            bidirectional=True,
            batch_first=True
        )
        
        self.out = nn.Linear(512, 1)
        
    def forward(self, x):
        
        # pass the inputs through the embedding layer
        x = self.embedding(x)
        
        # pass the embeddings through LSTM layer
        h_lstm, _ = self.lstm(x)
        
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        
        out = torch.cat((avg_pool, max_pool), 1)
        out = self.out(out)
        
        return out